# Imports

In [1]:
from architectures.lenet5 import LeNet5
from architectures.alexnet import AlexNet
from architectures.vgg16 import VGG16
from Datasets import Datasets
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras as keras


# Load Data

In [2]:
X_train, Y_train, X_test, Y_test = Datasets.load_cifar10()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255.
X_test = X_test/255.

_, x_size, y_size, n_filters = X_train.shape
_, classes = Y_train.shape

In [3]:
# For testing
X_train, _, Y_train, _ = train_test_split(X_train, Y_train, train_size=1024, test_size=1)

In [10]:
learning_rates = [(x ** 2) / 1000 if x != 0 else 0.001 for x in range(0, 11, 2)]
print(learning_rates)

[0.001, 0.004, 0.016, 0.036, 0.064, 0.1]


In [5]:
model = AlexNet(classes = classes, optimizer=keras.optimizers.SGD(learning_rate=0.1))

In [6]:
# Prepare data for cross validation
kf = KFold(n_splits=4) 
kf.get_n_splits(X_train)

for learning_rate in learning_rates:
    #Cross Validation loop
    for train_index, test_index in kf.split(X_train):

        # Build model
        model = VGG16(classes = classes, optimizer=keras.optimizers.SGD(learning_rate=learning_rate))

        X_tr = model.resize_images(X_train[train_index])
        Y_tr = Y_train[train_index]
        X_vl = model.resize_images(X_train[test_index])
        Y_vl = Y_train[test_index]

        # Generate extra data
        datagen = ImageDataGenerator(width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=0.2,
                        fill_mode='nearest',
                        horizontal_flip = True)
        datagen.fit(X_tr)
        
        # Create history
        batch_size = 8
        history = model.fit(datagen.flow(X_tr, Y_tr, batch_size=batch_size),
                              validation_data=(X_vl, Y_vl),
                              epochs=10, steps_per_epoch=len(X_tr)/batch_size)

        
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('LeNet5 ' + str(learning_rate) + ' learning rate model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('learning_rate')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()



TypeError: __init__() missing 1 required positional argument: 'input_shape'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64d23a20-b5bc-41cc-930c-3bd7ede12b0c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>